In [1]:
# Common modules
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, auc, roc_curve
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV
import scipy.sparse as sp
from sklearn.feature_selection import SelectKBest, mutual_info_classif

# knn specific modules
from sklearn.ensemble import RandomForestClassifier

In [2]:
BASE_PATH = os.path.join(os.getcwd(), os.pardir)
DATA_PATH = os.path.join(BASE_PATH, 'data')

In [3]:
train_df = pd.read_json(os.path.join(BASE_PATH, '01-milestone1', 'imputed_train.json'))
test_df = pd.read_json(os.path.join(DATA_PATH, 'test.json.zip'))

In [77]:
X_sparse = sp.load_npz(os.path.join(DATA_PATH, 'training_feats.npz'))

X_test = sp.load_npz(os.path.join(DATA_PATH, 'test_feats.npz'))

In [5]:
# X = train_df.drop(columns=['interest_level'])
y = train_df['interest_level']
y

10        medium
10000        low
100004      high
100007       low
100013       low
           ...  
9999      medium
99991        low
99992        low
99993        low
99994        low
Name: interest_level, Length: 49308, dtype: object

In [6]:
X_test

<74659x35522 sparse matrix of type '<class 'numpy.float64'>'
	with 3939246 stored elements in Compressed Sparse Row format>

In [7]:
X_sparse

<49308x35522 sparse matrix of type '<class 'numpy.float64'>'
	with 2626763 stored elements in Compressed Sparse Row format>

## Numerical features

In [101]:
X_org = train_df.drop(columns=['interest_level', 'building_id', 'created', 'description', 'display_address', 'features', 'manager_id', 'photos','street_address', 'listing_id'])

In [102]:
X_org

,bathrooms,bedrooms,latitude,longitude,price
10,1.5,3,40.7145,-73.9425,3000
10000,1.0,2,40.7947,-73.9667,5465
100004,1.0,1,40.7388,-74.0018,2850
100007,1.0,1,40.7539,-73.9677,3275
100013,1.0,4,40.8241,-73.9493,3350
...,...,...,...,...,...
9999,1.0,2,40.7426,-73.9790,3200
99991,1.0,1,40.7102,-74.0163,3950
99992,1.0,1,40.7601,-73.9900,2595
99993,1.0,0,40.7066,-74.0101,3350


In [103]:
test_df_num = test_df.drop(columns=['building_id', 'created', 'description', 'display_address', 'features', 'manager_id', 'photos','street_address','listing_id'])

In [104]:
test_df_num

,bathrooms,bedrooms,latitude,longitude,price
0,1.0,1,40.7185,-73.9865,2950
1,1.0,2,40.7278,-74.0000,2850
2,1.0,0,40.7260,-74.0026,2295
3,1.0,2,40.7321,-74.0028,2900
5,1.0,1,40.7054,-74.0095,3254
...,...,...,...,...,...
124003,1.0,1,40.7925,-73.9454,1700
124005,1.0,2,40.7456,-73.9797,4195
124006,1.0,0,40.7416,-73.9829,2400
124007,2.0,2,40.7485,-73.9800,6895


### Preprocessing

In [105]:
# Convert labels from {low, medium high} -> {0, 1, 2}
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

### Train-test split

We will do 5-fold cross-validation

In [106]:
seed = 36201431

In [107]:
model = RandomForestClassifier(n_estimators=100)

In [108]:
scores = cross_validate(model, X_org, y, cv=5, scoring=['neg_log_loss','accuracy'], return_train_score=True)

In [109]:
scores

{'fit_time': array([4.64845991, 4.79385495, 4.71716094, 4.58241701, 4.43578506]),
 'score_time': array([0.53607202, 0.55963421, 0.61079788, 0.545578  , 0.50530195]),
 'test_neg_log_loss': array([-1.24902489, -1.24716193, -1.36752084, -1.27430131, -1.32006799]),
 'train_neg_log_loss': array([-0.23591144, -0.23914936, -0.23757033, -0.23638905, -0.23617148]),
 'test_accuracy': array([0.68356484, 0.70029403, 0.69590347, 0.69432049, 0.69127789]),
 'train_accuracy': array([0.91813918, 0.91755609, 0.91786239, 0.91832286, 0.91827216])}

In [110]:
scores['test_neg_log_loss'].mean() * -1

1.2916153943156696

In [111]:
model.fit(X_org, y)
# model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [112]:
y_pred = model.predict_proba(test_df_num)

## Improving performance

In [113]:
model = RandomForestClassifier(n_estimators=300, min_samples_leaf=7)

In [114]:
scores = cross_validate(model, X_org, y, cv=5, scoring=['neg_log_loss','accuracy'], return_train_score=True)

In [115]:
scores
# test_scores = -1 * scores['test_neg_log_loss']
# train_scores = -1 * scores['train_neg_log_loss']
# test_score = test_scores.mean()
# train_score = train_scores.mean()
# print("Mean log loss test  score: {0}".format(test_score))
# print("Mean log loss train score: {0}".format(train_score))

{'fit_time': array([11.44549298, 10.17674685, 10.22130609, 10.29843521, 10.24684978]),
 'score_time': array([1.05726695, 1.07732105, 1.05375886, 1.06669903, 1.09448123]),
 'test_neg_log_loss': array([-0.65839142, -0.6493432 , -0.65551119, -0.64775672, -0.65763025]),
 'train_neg_log_loss': array([-0.51263735, -0.51508807, -0.51289334, -0.51489652, -0.51186523]),
 'test_accuracy': array([0.71154821, 0.72006489, 0.71800852, 0.70892495, 0.71419878]),
 'train_accuracy': array([0.77743694, 0.77378628, 0.77609897, 0.77611032, 0.77694687])}

In [116]:
model.fit(X_org, y)
# model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=7, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [117]:
y_pred = model.predict_proba(test_df_num)

## Categorical & Numerical features

### Preprocessing

In [55]:
# Convert labels from {low, medium high} -> {0, 1, 2}
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

In [78]:
#feature selection
kbest = SelectKBest(mutual_info_classif, k=10000).fit(X_sparse,y)
X_fseltd = kbest.transform(X_sparse) 

In [79]:
X_fseltd

<49308x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 2335631 stored elements in Compressed Sparse Row format>

In [80]:
X_test = kbest.transform(X_test)


In [81]:
X_test

<74659x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 3517152 stored elements in Compressed Sparse Row format>

### Train-test split

We will do 5-fold cross-validation

In [70]:
seed = 36201431

In [82]:
model = RandomForestClassifier(n_estimators=200, min_samples_leaf=5)

In [83]:
# scores = cross_validate(model, X_fseltd, y, cv=5, scoring=['neg_log_loss','accuracy'])
scores = cross_validate(model, X_fseltd, y, cv=5, scoring=['neg_log_loss','accuracy'], return_train_score=True)

In [84]:
scores
# test_scores = -1 * scores['test_neg_log_loss']
# train_scores = -1 * scores['train_neg_log_loss']
# test_score = test_scores.mean()
# train_score = train_scores.mean()
# print("Mean log loss test  score: {0}".format(test_score))
# print("Mean log loss train score: {0}".format(train_score))

{'fit_time': array([42.63136816, 37.6082828 , 42.57876778, 43.32008791, 39.78621912]),
 'score_time': array([1.12649703, 1.50175714, 1.11025095, 1.13014603, 1.12656093]),
 'test_neg_log_loss': array([-0.7069493 , -0.70172152, -0.69925162, -0.6962809 , -0.70610336]),
 'train_neg_log_loss': array([-0.6483763 , -0.65189201, -0.65215472, -0.65280364, -0.65293627]),
 'test_accuracy': array([0.69552874, 0.69603569, 0.69590347, 0.69624746, 0.69584178]),
 'train_accuracy': array([0.69785778, 0.69760426, 0.69707955, 0.69727236, 0.69772866])}

In [85]:
model.fit(X_fseltd, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [86]:
y_pred = model.predict_proba(X_test)

In [87]:
le.classes_

array(['high', 'low', 'medium'], dtype=object)

## Make submission file for kaggle

In [30]:
n = 2

In [22]:
def create_submission_csv(y_pred, X_test_indexes):
    df = pd.DataFrame(y_pred, columns=le.classes_)
    df.index = X_test_indexes
    df.index.name = 'listing_id'
    return df

In [118]:
X_test_indexes = test_df['listing_id']
output = create_submission_csv(y_pred, X_test_indexes)
csv_name = 'rf_predictions_{}.csv'.format(n)
output.to_csv(csv_name)
n += 1